# MVP: Modelagem de Ameaças com IA (STRIDE)

Este notebook cria um MVP para detectar componentes em diagramas de arquitetura e gerar ameaças e contramedidas baseadas em STRIDE.

## 1) Setup

In [ ]:
!apt-get -qq update && apt-get -qq install -y tesseract-ocr
!pip -q install ultralytics opencv-python pytesseract

In [ ]:
from pathlib import Path
import zipfile
from google.colab import files
import cv2
import pytesseract
from ultralytics import YOLO

## 2) Escolha do modo
Use `treino` quando tiver dataset anotado (YOLO/COCO).
Use `inferencia` quando tiver apenas uma imagem do diagrama.

In [ ]:
MODO = 'inferencia'  # 'treino' ou 'inferencia'
print('Modo selecionado:', MODO)

## 3A) Upload do dataset (somente treino)
Envie um .zip contendo a estrutura YOLO:
- images/train
- images/val
- labels/train
- labels/val

In [ ]:
if MODO != 'treino':
    print('Pulando upload de dataset (modo não é treino).')
else:
    ARQUIVOS = files.upload()
    NOME_ZIP = next(iter(ARQUIVOS))

    DIR_DADOS = Path('/content/data')
    DIR_DADOS.mkdir(parents=True, exist_ok=True)

    with zipfile.ZipFile(NOME_ZIP, 'r') as zf:
        zf.extractall(DIR_DADOS)

    print('Dataset extraído em:', DIR_DADOS)

    # Validação automática da estrutura do dataset
    EXTENSOES_IMAGEM = {'.jpg', '.jpeg', '.png', '.bmp'}

    def ajustar_raiz_dataset(dir_base):
        if (dir_base / 'images').exists():
            return dir_base
        subdirs = [p for p in dir_base.iterdir() if p.is_dir()]
        if len(subdirs) == 1 and (subdirs[0] / 'images').exists():
            return subdirs[0]
        return dir_base

    def listar_imagens(pasta):
        return [p for p in pasta.rglob('*') if p.suffix.lower() in EXTENSOES_IMAGEM]

    def validar_estrutura(dir_base):
        esperados = [
            dir_base / 'images' / 'train',
            dir_base / 'images' / 'val',
            dir_base / 'labels' / 'train',
            dir_base / 'labels' / 'val',
        ]
        faltando = [str(p) for p in esperados if not p.exists()]
        if faltando:
            raise FileNotFoundError('Pastas ausentes: ' + ', '.join(faltando))

        resumo = {}
        for split in ['train', 'val']:
            imgs = listar_imagens(dir_base / 'images' / split)
            labels = list((dir_base / 'labels' / split).rglob('*.txt'))
            labels_set = {p.stem for p in labels}
            sem_label = [p for p in imgs if p.stem not in labels_set]
            resumo[split] = {
                'imagens': len(imgs),
                'labels': len(labels),
                'sem_label': len(sem_label)
            }
        return resumo

    DIR_DADOS = ajustar_raiz_dataset(DIR_DADOS)
    print('Diretório de dados (ajustado):', DIR_DADOS)

    resumo = validar_estrutura(DIR_DADOS)
    print('Validação OK. Resumo:', resumo)

## 3B) Upload de imagem única (somente inferência)
Envie uma imagem (.png ou .jpg) do diagrama de arquitetura.

In [ ]:
if MODO != 'inferencia':
    print('Pulando upload de imagem (modo não é inferência).')
else:
    ARQUIVOS_IMG = files.upload()
    NOME_IMG = next(iter(ARQUIVOS_IMG))
    CAMINHO_IMAGEM = str(Path(NOME_IMG).resolve())
    print('Imagem carregada:', CAMINHO_IMAGEM)

## 4) Treinamento do detector (YOLOv8) — opcional

In [ ]:
PESOS_BASE = 'yolov8n.pt'
modelo = YOLO(PESOS_BASE)

if MODO == 'treino':
    CAMINHO_DATA_YAML = '/content/data/data.yaml'  # ajuste conforme seu zip
    # modelo.train(data=CAMINHO_DATA_YAML, epochs=50, imgsz=640)

## 5) Inferência

In [ ]:
if MODO == 'inferencia':
    PESOS_INFERENCIA = 'yolov8n.pt'  # ou '/content/runs/detect/train/weights/best.pt'
    modelo = YOLO(PESOS_INFERENCIA)

    resultados = modelo.predict(source=CAMINHO_IMAGEM, conf=0.25)
    deteccoes = []
    for r in resultados:
        for c in r.boxes.cls.tolist():
            deteccoes.append(modelo.names[int(c)])
    print('Detecções:', deteccoes)

## 6) Mapeamento STRIDE

In [ ]:
MAPA_STRIDE = {
    'user': ['Spoofing', 'Elevation of Privilege'],
    'auth': ['Spoofing', 'Elevation of Privilege', 'Repudiation'],
    'api': ['Tampering', 'Repudiation', 'Denial of Service'],
    'gateway': ['Tampering', 'Repudiation', 'Denial of Service'],
    'database': ['Information Disclosure', 'Tampering'],
    'storage': ['Information Disclosure', 'Tampering'],
    'queue': ['Tampering', 'Denial of Service'],
    'frontend': ['Spoofing', 'Tampering', 'Information Disclosure'],
    'backend': ['Tampering', 'Repudiation', 'Denial of Service'],
    'network': ['Spoofing', 'Information Disclosure', 'Denial of Service'],
    'server': ['Tampering', 'Denial of Service', 'Elevation of Privilege'],
    'third_party': ['Spoofing', 'Information Disclosure', 'Repudiation']
}

def mapear_stride(componentes):
    saida = {}
    for c in componentes:
        saida[c] = MAPA_STRIDE.get(c, [])
    return saida

## 7) Geração de relatório

In [ ]:
componentes = []
if 'deteccoes' in globals() and deteccoes:
    componentes = deteccoes
else:
    componentes = ['api', 'database', 'user']

mapeamento = mapear_stride(componentes)

linhas = ['# Relatório de Modelagem de Ameaças', '']
for comp, ameacas in mapeamento.items():
    linhas.append(f'## {comp}')
    if ameacas:
        for a in ameacas:
            linhas.append(f'- {a}')
    else:
        linhas.append('- Nenhuma ameaça mapeada')
    linhas.append('')

relatorio = '\n'.join(linhas)
print(relatorio)

with open('relatorio.md', 'w', encoding='utf-8') as f:
    f.write(relatorio)